In [ ]:
import tensorflow as tf
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import pathlib
import PIL
import PIL.Image

In [ ]:
data_dir = pathlib.Path('./data')
data = list(data_dir.glob('*/*.*'))

In [ ]:
batch_size = 17
img_height = 227
img_width = 227



train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
print(train_ds)

In [ ]:
base_model = tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False,
        pooling='max',
        input_shape=(img_height, img_width, 3),
        weights='imagenet')

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(512, 'relu'),
    tf.keras.layers.Dense(256, 'relu'),
    tf.keras.layers.Dense(2, 'softmax')
])
    
model.summary()

In [ ]:
snapshot_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="./snapshots",
    verbose=1)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

history = model.fit(
    train_ds,
    steps_per_epoch=2, #use way more steps here: number of samples / batch size
    epochs=30 #use way more or use EarlyStopping callback
)